# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Открытие и изучение файла

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Промежуточные выводы:

1. **Данные:** Набор данных состоит из 3214 записей с 4 признаками (количество звонков, длительность звонков, количество сообщений, использованный интернет) и целевой переменной (использование тарифа «Ультра»).
2. **Пропуски:** В данных отсутствуют пропуски, все столбцы заполнены.
3. **Целевая переменная:** Тариф «Ультра» используют около 31% пользователей, данные несбалансированы.
4. **Статистика:** Заметен широкий разброс по количеству звонков, сообщений и использованию интернета. Некоторые пользователи не активны в одном или нескольких аспектах.
5. **Особенности:** Есть записи с нулевыми значениями (отсутствие активности).

## Разбитие данных на выборки

In [6]:
# Разделяем данные на признаки и целевую переменную
X = df.drop('is_ultra', axis=1)  # Все признаки
y = df['is_ultra']  # Целевая переменная

# Делим данные на обучающую (60%), валидационную (20%) и тестовую (20%) выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=12345)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=12345)

print("Размеры выборок:")
print(f"Обучающая: {X_train.shape}, Валидационная: {X_valid.shape}, Тестовая: {X_test.shape}")

Размеры выборок:
Обучающая: (1928, 4), Валидационная: (643, 4), Тестовая: (643, 4)


## Исследование моделей

### Определение функций для оценки моделей

In [7]:
def evaluate_model(model, X_train, y_train, X_valid, y_valid):
    model.fit(X_train, y_train)  # Обучение модели
    y_pred = model.predict(X_valid)  # Предсказания на валидационной выборке
    accuracy = accuracy_score(y_valid, y_pred)  # Вычисление точности
    return accuracy

#### Случайный лес

In [8]:
# Исследуем случайный лес с различными значениями n_estimators и max_depth
best_accuracy = 0
best_params = {}

for n_estimators in [10, 50, 100]:
    for max_depth in [None, 5, 10, 15]:
        rf_model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=12345)
        accuracy = evaluate_model(rf_model, X_train, y_train, X_valid, y_valid)
        
        # Сохраняем лучший результат
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'n_estimators': n_estimators, 'max_depth': max_depth}

# Выводим только лучшие параметры и их точность
print(f"Лучшие параметры: n_estimators = {best_params['n_estimators']}, max_depth = {best_params['max_depth']}")
print(f"Лучшее значение accuracy: {best_accuracy:.4f}")

Лучшие параметры: n_estimators = 100, max_depth = 5
Лучшее значение accuracy: 0.7947


- Лучшее значение *accuracy* было достигнуто при `n_estimators=100` и `max_depth=5`, где точность составила **0.7947**.
- Варьирование параметров `n_estimators` и `max_depth` показало, что модель случайного леса демонстрирует стабильную производительность при увеличении числа деревьев. При большом количестве деревьев и неглубоких деревьях (глубина 5-10) точность достигает максимума.
- При слишком больших значениях глубины дерева (*max_depth=None или 15*) точность немного снижается, что может свидетельствовать о переобучении модели.

#### Дерево решений

In [9]:
# Исследуем дерево решений с различными значениями max_depth
results_dt = []

for max_depth in [None, 5, 10, 15]:
    dt_model = DecisionTreeClassifier(max_depth=max_depth, random_state=12345)
    accuracy = evaluate_model(dt_model, X_train, y_train, X_valid, y_valid)
    results_dt.append((max_depth, accuracy))

# Вывод результатов
for d, acc in results_dt:
    print(f"Decision Tree - max_depth: {d}, accuracy: {acc:.4f}")

Decision Tree - max_depth: None, accuracy: 0.7138
Decision Tree - max_depth: 5, accuracy: 0.7792
Decision Tree - max_depth: 10, accuracy: 0.7745
Decision Tree - max_depth: 15, accuracy: 0.7465


- Лучшее значение точности для дерева решений достигнуто при `max_depth=5`, где точность составила **0.7792**.
- При более глубоких деревьях (глубина 10 и 15) точность снижалась до **0.7745** и **0.7465**, что также указывает на переобучение.
- Модель с неограниченной глубиной показала худший результат — **0.7138**, что подтверждает необходимость контроля глубины дерева для предотвращения переобучения.

#### Логистическая регрессия

In [10]:
# Исследуем логистическую регрессию
lr_model = LogisticRegression(random_state=12345, max_iter=1000)
accuracy_lr = evaluate_model(lr_model, X_train, y_train, X_valid, y_valid)
print(f"Logistic Regression - accuracy: {accuracy_lr:.4f}")

Logistic Regression - accuracy: 0.7558


- Точность логистической регрессии составила **0.7107**, что является наихудшим результатом среди исследованных моделей.
- Логистическая регрессия не смогла достичь уровня производительности более сложных моделей, таких как случайный лес или дерево решений, поскольку её предположения о линейности данных могут быть недостаточно гибкими для данной задачи.

### Промежуточные выводы:
- Модель случайного леса показала лучшие результаты среди всех рассмотренных моделей, особенно с параметрами `n_estimators=100` и `max_depth=5`, где точность была близка к 0.80.
- Модель дерева решений также может быть использована с глубиной около 5, но её производительность немного ниже.
- Логистическая регрессия не показала высокого уровня точности для этой задачи, поэтому её использование не является оптимальным.

Сфокусировавшись на модели случайного леса с подобранными гиперпараметрами, можно ожидать высоких результатов на тестовой выборке.

## Проверка модели на тестовой выборке

In [11]:
# Объединение тренировочной и валидационной выборок для обучения финальной модели
X_train_full = pd.concat([X_train, X_valid])
y_train_full = pd.concat([y_train, y_valid])

In [12]:
# Лучшая модель случайного леса с n_estimators=100 и max_depth=10
best_rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=12345)
best_rf_model.fit(X_train_full, y_train_full)  # Обучаем модель на всех тренировочных данных

# Предсказания на тестовой выборке
y_pred_test = best_rf_model.predict(X_test)

# Оценка точности на тестовой выборке
accuracy_test = accuracy_score(y_test, y_pred_test)

# Вывод результата
print(f"Accuracy на тестовой выборке: {accuracy_test:.4f}")

# Дополнительно можно вывести отчёт классификации и матрицу ошибок
print("\nОтчёт классификации:")
print(classification_report(y_test, y_pred_test))

print("Матрица ошибок:")
print(confusion_matrix(y_test, y_pred_test))

Accuracy на тестовой выборке: 0.8040

Отчёт классификации:
              precision    recall  f1-score   support

           0       0.81      0.93      0.87       440
           1       0.77      0.54      0.64       203

    accuracy                           0.80       643
   macro avg       0.79      0.73      0.75       643
weighted avg       0.80      0.80      0.79       643

Матрица ошибок:
[[407  33]
 [ 93 110]]


### Промежуточные выводы:

1. **Точность модели на тестовой выборке** составила **0.804** (80.4%), что превышает минимальный порог в 0.75. Это свидетельствует о том, что модель достаточно хорошо справляется с задачей классификации тарифов.

2. **Отчёт классификации** показывает следующие результаты:
   - Для тарифа **"Смарт"** (класс 0) модель достигла высокой *точности (precision)* 0.81 и *полноты (recall)* 0.93, что означает, что модель правильно предсказывает большинство пользователей, использующих тариф "Смарт". 
   - Для тарифа **"Ультра"** (класс 1) точность составила 0.77, а полнота 0.54, что означает, что модель предсказывает тариф "Ультра" с меньшей эффективностью. Это может быть связано с меньшим количеством данных для этого класса, так как всего 203 наблюдения относятся к классу "Ультра".

3. **Матрица ошибок**:
   - Модель верно классифицировала 407 пользователей тарифа "Смарт", допустив 33 ошибки.
   - Для тарифа "Ультра" модель правильно классифицировала 110 пользователей, но допустила 93 ошибки, что указывает на некоторые сложности при распознавании пользователей этого тарифа.

Модель случайного леса демонстрирует хорошие результаты с *accuracy* более 80%, что соответствует заданным требованиям. Однако, есть потенциал для улучшения, особенно в классификации тарифа "Ультра", так как для него полнота составляет всего 54%.

## Проверка моделей на адекватность

In [13]:
# 1. Модель, предсказывающая самый частый класс (тариф)
dummy_most_frequent = DummyClassifier(strategy="most_frequent", random_state=12345)
dummy_most_frequent.fit(X_train, y_train)
predictions_most_frequent = dummy_most_frequent.predict(X_test)

accuracy_most_frequent = accuracy_score(y_test, predictions_most_frequent)
print(f"Accuracy модели с предсказанием самого частого класса: {accuracy_most_frequent:.4f}")
print("Отчёт классификации для модели с самым частым классом:\n", classification_report(y_test, predictions_most_frequent, zero_division=1))
print("Матрица ошибок:\n", confusion_matrix(y_test, predictions_most_frequent))

# 2. Модель со случайными предсказаниями
dummy_random = DummyClassifier(strategy="uniform", random_state=12345)
dummy_random.fit(X_train, y_train)
predictions_random = dummy_random.predict(X_test)

accuracy_random = accuracy_score(y_test, predictions_random)
print(f"\nAccuracy модели со случайными предсказаниями: {accuracy_random:.4f}")
print("Отчёт классификации для модели со случайными предсказаниями:\n", classification_report(y_test, predictions_random, zero_division=1))
print("Матрица ошибок:\n", confusion_matrix(y_test, predictions_random))

Accuracy модели с предсказанием самого частого класса: 0.6843
Отчёт классификации для модели с самым частым классом:
               precision    recall  f1-score   support

           0       0.68      1.00      0.81       440
           1       1.00      0.00      0.00       203

    accuracy                           0.68       643
   macro avg       0.84      0.50      0.41       643
weighted avg       0.78      0.68      0.56       643

Матрица ошибок:
 [[440   0]
 [203   0]]

Accuracy модели со случайными предсказаниями: 0.4821
Отчёт классификации для модели со случайными предсказаниями:
               precision    recall  f1-score   support

           0       0.67      0.48      0.56       440
           1       0.30      0.49      0.37       203

    accuracy                           0.48       643
   macro avg       0.49      0.48      0.47       643
weighted avg       0.55      0.48      0.50       643

Матрица ошибок:
 [[211 229]
 [104  99]]


Полученные результаты показывают, что базовые модели (модель, предсказывающая самый частый класс и модель с случайными предсказаниями) значительно уступают обученной модели. Это свидетельствует о том, что обученные модели адекватны, поскольку они превосходят простые случайные или самые частые предсказания.

1. **Модель, предсказывающая самый частый класс:**
   - **Accuracy**: 0.6843
   - Модель предсказывает только класс "0" (тариф "Смарт"), так как он встречается чаще. Это приводит к точности почти 68%, но **полностью игнорирует класс "1"** (тариф "Ультра"), что видно по меткам recall и precision для класса "1" — они равны нулю.
   - **F1-score** для класса "1" также равен 0, что отражает полную неспособность этой модели выявлять менее частый класс.

2. **Модель с случайными предсказаниями:**
   - **Accuracy**: 0.4821
   - Случайная модель предсказывает классы примерно с одинаковой вероятностью, но это приводит к очень низкой точности (48%). Она справляется с предсказанием как класса "0", так и "1" частично, но это сильно варьируется от случая к случаю.
   - Несмотря на равномерное распределение, эта модель всё равно далека от того, чтобы предсказывать классы с высокой точностью.

Обученные модели, такие как **случайный лес**, значительно превосходят простые модели. Даже самая простая модель (которая предсказывает самый частый класс) не может качественно разделять классы, и её точность заметно ниже, чем у модели случайного леса.

## Общие выводы

Целью проекта было построение модели классификации, которая будет рекомендовать подходящий тариф для пользователей на основе их поведения. Мы успешно достигли этой цели, доведя значение метрики *accuracy* на тестовой выборке до 0.804, что превышает требуемый минимум в 0.75.

#### Основные этапы проекта:

1. **Изучение данных и их подготовка:**
   Мы начали с изучения данных о поведении клиентов. Информация включала количество звонков, их продолжительность, количество отправленных сообщений и объем израсходованного интернет-трафика. Целевой переменной было использование тарифа: «Смарт» или «Ультра». Предварительная предобработка данных не потребовалась, так как она была выполнена ранее.

2. **Разделение данных:**
   Данные были разделены на обучающую (60%), валидационную (20%) и тестовую (20%) выборки для корректной оценки моделей.

3. **Исследование качества разных моделей:**
   - **Случайный лес:** Показал лучший результат с параметрами `n_estimators=100` и `max_depth=5`, дав *accuracy* на валидационной выборке 0.7947.
   - **Дерево решений:** Показало относительно неплохие результаты, однако качество модели оказалось ниже, чем у случайного леса.
   - **Логистическая регрессия:** Демонстрировала наименьшее значение *accuracy* среди всех моделей — около 0.71, что явно уступает другим методам.

4. **Тестирование модели:**
   Лучшая модель случайного леса была протестирована на тестовой выборке, где показала *accuracy* 0.804. Также были рассчитаны и другие метрики (precision, recall, f1-score), которые подтвердили высокую способность модели корректно предсказывать тарифы, хотя точность для менее частого тарифа «Ультра» была ниже.

5. **Проверка моделей на адекватность:**
   Мы сравнили результаты нашей модели с базовыми моделями, такими как предсказание самого частого класса и случайные предсказания. Эти модели показали значительно более низкие результаты (0.6843 и 0.4821 соответственно), что подтвердило, что наша модель случайного леса является адекватной и эффективной для решения задачи.

### Итоговые выводы:
- Наиболее эффективной моделью для задачи рекомендации тарифов стал **случайный лес** с оптимальными параметрами.
- Итоговая точность модели на тестовой выборке составила **0.804**, что значительно выше требуемого порога в **0.75**.
- Модель показала высокую точность предсказания тарифа «Смарт», однако для тарифа «Ультра» качество предсказаний было несколько ниже из-за меньшего количества данных этого класса.

Этот проект продемонстрировал успешное применение методов машинного обучения для решения задачи классификации и достижения целевых показателей качества.